In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

--2020-05-29 09:09:18--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  14.4MB/s    in 0.3s    

2020-05-29 09:09:19 (14.4 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [4]:
%cd ml-100k
!shuf ua.base -o ua.base.aftershuffled
!head -10 ua.base.aftershuffled

[Errno 2] No such file or directory: 'ml-100k'
/home/ec2-user/SageMaker/ml-100k
676	117	4	892686244
566	327	3	881649273
663	287	5	889492720
133	300	3	890588577
1	7	4	875071561
876	22	4	879428451
715	944	2	875963755
825	273	5	880756401
831	156	4	891354751
313	391	3	891028360


In [5]:
!head -10 ua.test

1	20	4	887431883
1	33	4	878542699
1	61	4	878542420
1	117	3	874965739
1	155	2	878542201
1	160	4	875072547
1	171	5	889751711
1	189	3	888732928
1	202	5	875072442
1	265	4	878542441


In [6]:
import boto3, csv, io, json
import numpy as np
from scipy.sparse import lil_matrix

from sagemaker import get_execution_role
from sagemaker.predictor import json_deserializer

import sagemaker
import sagemaker.amazon.common as smac

In [7]:
numUsers=943
numMovies=1682
numFeatures= numUsers + numMovies
numRatingsTrain=90570
numRatingsTest=9430

In [10]:
moviesRated = {}
for userId in range(numUsers):
    moviesRated[str(userId)]=[]
 
with open('ua.base.aftershuffled','r') as f:
    samples=csv.reader(f,delimiter='\t')
    for userId,movieId,rating,timestamp in samples:
        moviesRated[str(int(userId)-1)].append(int(movieId)-1)
print(moviesRated)

{'0': [6, 103, 227, 151, 178, 242, 46, 197, 12, 166, 13, 169, 102, 235, 20, 106, 28, 30, 121, 183, 268, 253, 97, 191, 270, 173, 256, 118, 91, 33, 208, 25, 53, 200, 185, 140, 128, 70, 75, 225, 8, 195, 142, 47, 23, 108, 50, 89, 39, 15, 210, 175, 63, 232, 234, 230, 134, 112, 77, 44, 202, 10, 34, 192, 62, 143, 120, 247, 55, 104, 133, 189, 80, 95, 74, 101, 21, 122, 211, 16, 212, 246, 263, 252, 59, 7, 35, 203, 51, 223, 111, 4, 255, 40, 9, 165, 81, 45, 209, 24, 65, 180, 164, 249, 217, 198, 141, 114, 127, 79, 186, 168, 150, 84, 48, 61, 18, 37, 113, 66, 204, 218, 149, 99, 5, 119, 228, 221, 123, 135, 207, 187, 163, 220, 155, 233, 184, 214, 266, 251, 107, 243, 36, 109, 167, 245, 267, 117, 257, 190, 2, 57, 162, 83, 93, 160, 148, 261, 85, 219, 231, 11, 258, 271, 110, 3, 158, 138, 259, 136, 181, 193, 92, 216, 139, 153, 88, 177, 1, 224, 147, 172, 14, 137, 67, 244, 182, 96, 144, 42, 269, 157, 241, 229, 124, 22, 49, 206, 196, 87, 156, 43, 115, 56, 222, 237, 161, 72, 38, 100, 132, 265, 146, 58, 17, 86, 

In [11]:
def loadDataset(filename, line, column):
    X = lil_matrix((line, column)).astype('float32')
    Y = []
    l = 0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter='\t')
        for userId,movieId,rating,timestamp in samples:
            X[l,int(userId)-1] = 1
            X[l,int(numUsers)+int(movieId)-1] = 1
            if int(rating) >= 4:
                Y.append(1)
            else:
                Y.append(0)
            l=l+1
            
    Y = np.array(Y).astype('float32')
    return X,Y

In [12]:
X_training, Y_training = loadDataset('ua.base.aftershuffled', numRatingsTrain, numFeatures)
X_test, Y_test = loadDataset('ua.test',numRatingsTest,numFeatures)

In [13]:
print(X_training.shape)
print(Y_training.shape)
assert X_training.shape == (numRatingsTrain, numFeatures)
assert Y_training.shape == (numRatingsTrain, )
zeros = np.count_nonzero(Y_training)
print("Training: %d nhãn 0, %d nhãn 1" % (zeros, numRatingsTrain - zeros))

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (numRatingsTest, numFeatures)
assert Y_test.shape  == (numRatingsTest, )
zeros = np.count_nonzero(Y_test)
print("Test: %d nhãn 0, %d nhãn 1" % (zeros, numRatingsTest-zeros))

(90570, 2625)
(90570,)
Training: 49906 nhãn 0, 40664 nhãn 1
(9430, 2625)
(9430,)
Test: 5469 nhãn 0, 3961 nhãn 1


In [14]:
bucket = 'bukmovie'
prefix = 'sagemaker/Movie-Recommender'

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'trainset')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'testset')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [16]:
def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buffer = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buffer, X, Y)
    buffer.seek(0)
    objec = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(objec).upload_fileobj(buffer)
    return 's3://{}/{}'.format(bucket,objec)
    
train_data = writeDatasetToProtobuf(X_training, Y_training, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key)    
  
print(train_data)
print(test_data)
print('Output: {}'.format(output_prefix))

s3://bukmovie/sagemaker/Movie-Recommender/trainset/train.protobuf
s3://bukmovie/sagemaker/Movie-Recommender/testset/test.protobuf
Output: s3://bukmovie/sagemaker/Movie-Recommender/output


In [17]:
containers = {'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest'}

In [18]:
fm_model = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                   get_execution_role(), 
                                   train_instance_count=1, 
                                   train_instance_type='ml.c4.xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm_model.set_hyperparameters(feature_dim=numFeatures,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=100)

fm_model.fit({'train': train_data, 'test': test_data})

2020-05-29 09:36:34 Starting - Starting the training job...
2020-05-29 09:36:36 Starting - Launching requested ML instances.........
2020-05-29 09:38:18 Starting - Preparing the instances for training......
2020-05-29 09:39:26 Downloading - Downloading input data
2020-05-29 09:39:26 Training - Downloading the training image......
2020-05-29 09:40:35 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python2.7/site-packages/pandas/util/nosetester.py:13: DeprecationWarning: Importing from numpy.testing.nosetester is deprecated, import from numpy.testing instead.
  from numpy.testing import nosetester
[05/29/2020 09:40:38 INFO 139926618736448] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers'

[2020-05-29 09:40:43.003] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 14, "duration": 630, "num_examples": 91, "num_bytes": 5796480}
[05/29/2020 09:40:43 INFO 139926618736448] #quality_metric: host=algo-1, epoch=6, train binary_classification_accuracy <score>=0.685505494505
[05/29/2020 09:40:43 INFO 139926618736448] #quality_metric: host=algo-1, epoch=6, train binary_classification_cross_entropy <loss>=0.632891014686
[05/29/2020 09:40:43 INFO 139926618736448] #quality_metric: host=algo-1, epoch=6, train binary_f_1.000 <score>=0.756858247313
#metrics {"Metrics": {"update.time": {"count": 1, "max": 632.457971572876, "sum": 632.457971572876, "min": 632.457971572876}}, "EndTime": 1590745243.003815, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1590745242.370544}

[05/29/2020 09:40:43 INFO 139926618736448] #progress_metric: host=algo-1, completed 7 % of epochs
#metrics {"Metrics": {"Max

[2020-05-29 09:40:53.441] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 48, "duration": 588, "num_examples": 91, "num_bytes": 5796480}
[05/29/2020 09:40:53 INFO 139926618736448] #quality_metric: host=algo-1, epoch=23, train binary_classification_accuracy <score>=0.729802197802
[05/29/2020 09:40:53 INFO 139926618736448] #quality_metric: host=algo-1, epoch=23, train binary_classification_cross_entropy <loss>=0.568598597265
[05/29/2020 09:40:53 INFO 139926618736448] #quality_metric: host=algo-1, epoch=23, train binary_f_1.000 <score>=0.768609662909
#metrics {"Metrics": {"update.time": {"count": 1, "max": 590.8920764923096, "sum": 590.8920764923096, "min": 590.8920764923096}}, "EndTime": 1590745253.442102, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1590745252.850441}

[05/29/2020 09:40:53 INFO 139926618736448] #progress_metric: host=algo-1, completed 24 % of epochs
#metrics {"Metrics"

[2020-05-29 09:41:03.147] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 80, "duration": 601, "num_examples": 91, "num_bytes": 5796480}
[05/29/2020 09:41:03 INFO 139926618736448] #quality_metric: host=algo-1, epoch=39, train binary_classification_accuracy <score>=0.735340659341
[05/29/2020 09:41:03 INFO 139926618736448] #quality_metric: host=algo-1, epoch=39, train binary_classification_cross_entropy <loss>=0.545733835744
[05/29/2020 09:41:03 INFO 139926618736448] #quality_metric: host=algo-1, epoch=39, train binary_f_1.000 <score>=0.769742628781
#metrics {"Metrics": {"update.time": {"count": 1, "max": 603.4281253814697, "sum": 603.4281253814697, "min": 603.4281253814697}}, "EndTime": 1590745263.147876, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1590745262.543557}

[05/29/2020 09:41:03 INFO 139926618736448] #progress_metric: host=algo-1, completed 40 % of epochs
#metrics {"Metrics"

[2020-05-29 09:41:13.546] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 114, "duration": 577, "num_examples": 91, "num_bytes": 5796480}
[05/29/2020 09:41:13 INFO 139926618736448] #quality_metric: host=algo-1, epoch=56, train binary_classification_accuracy <score>=0.738208791209
[05/29/2020 09:41:13 INFO 139926618736448] #quality_metric: host=algo-1, epoch=56, train binary_classification_cross_entropy <loss>=0.532279374343
[05/29/2020 09:41:13 INFO 139926618736448] #quality_metric: host=algo-1, epoch=56, train binary_f_1.000 <score>=0.770868799954
#metrics {"Metrics": {"update.time": {"count": 1, "max": 579.4360637664795, "sum": 579.4360637664795, "min": 579.4360637664795}}, "EndTime": 1590745273.546895, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1590745272.966617}

[05/29/2020 09:41:13 INFO 139926618736448] #progress_metric: host=algo-1, completed 57 % of epochs
#metrics {"Metrics

[2020-05-29 09:41:23.185] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 146, "duration": 613, "num_examples": 91, "num_bytes": 5796480}
[05/29/2020 09:41:23 INFO 139926618736448] #quality_metric: host=algo-1, epoch=72, train binary_classification_accuracy <score>=0.745461538462
[05/29/2020 09:41:23 INFO 139926618736448] #quality_metric: host=algo-1, epoch=72, train binary_classification_cross_entropy <loss>=0.524111387148
[05/29/2020 09:41:23 INFO 139926618736448] #quality_metric: host=algo-1, epoch=72, train binary_f_1.000 <score>=0.775936620331
#metrics {"Metrics": {"update.time": {"count": 1, "max": 615.7290935516357, "sum": 615.7290935516357, "min": 615.7290935516357}}, "EndTime": 1590745283.185859, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1590745282.569189}

[05/29/2020 09:41:23 INFO 139926618736448] #progress_metric: host=algo-1, completed 73 % of epochs
#metrics {"Metrics

[2020-05-29 09:41:33.460] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 180, "duration": 589, "num_examples": 91, "num_bytes": 5796480}
[05/29/2020 09:41:33 INFO 139926618736448] #quality_metric: host=algo-1, epoch=89, train binary_classification_accuracy <score>=0.747714285714
[05/29/2020 09:41:33 INFO 139926618736448] #quality_metric: host=algo-1, epoch=89, train binary_classification_cross_entropy <loss>=0.517754020942
[05/29/2020 09:41:33 INFO 139926618736448] #quality_metric: host=algo-1, epoch=89, train binary_f_1.000 <score>=0.777805737292
#metrics {"Metrics": {"update.time": {"count": 1, "max": 591.4609432220459, "sum": 591.4609432220459, "min": 591.4609432220459}}, "EndTime": 1590745293.460922, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1590745292.868647}

[05/29/2020 09:41:33 INFO 139926618736448] #progress_metric: host=algo-1, completed 90 % of epochs
#metrics {"Metrics


2020-05-29 09:41:49 Uploading - Uploading generated training model
2020-05-29 09:41:49 Completed - Training job completed
Training seconds: 156
Billable seconds: 156


In [19]:
fm_predictor = fm_model.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1)

-------------!

In [20]:
def fm_serializer(data):
    json_data = {'instances': []}
    for row in data:
        json_data['instances'].append({'features': row.tolist()})
    #print js
    return json.dumps(json_data)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

In [22]:
result = fm_predictor.predict(X_test[9100].toarray())
print(result)
print (Y_test[9100])

{'predictions': [{'score': 0.8356688022613525, 'predicted_label': 1.0}]}
0.0
